In [10]:
import pandas as pd
import gdown
import json
import os
import numpy as np

print("🚀 DÉMARRAGE : Mode Strict (Films >= 1960)...")

# ==============================================================================
# ÉTAPE 1 : TMDB (Fichier Drive)
# ==============================================================================
print("\n1️⃣  TMDB : Chargement et Nettoyage...")
id_drive = '1VB5_gl1fnyBDzcIOXZ5vUSbCY68VZN1v'
output_tmdb = 'tmdb_final.csv'
url_drive = f'https://drive.google.com/uc?id={id_drive}'

if not os.path.exists(output_tmdb):
    print("   -> Téléchargement...")
    gdown.download(url_drive, output_tmdb, quiet=False)
else:
    print("   -> Fichier déjà là.")

df_tmdb = pd.read_csv(output_tmdb)

# Nettoyage JSON
def clean_json(x):
    try:
        if pd.isna(x): return np.nan
        data = json.loads(x.replace("'", '"'))
        return ", ".join([i['name'] for i in data])
    except:
        return np.nan

if 'production_companies' in df_tmdb.columns:
    df_tmdb['companies_clean'] = df_tmdb['production_companies'].apply(clean_json)

# Suppression des colonnes inutiles
cols_drop = ['homepage', 'video', 'backdrop_path', 'status', 'production_companies', 'production_countries']
df_tmdb = df_tmdb.drop(columns=[c for c in cols_drop if c in df_tmdb.columns])


# ==============================================================================
# ÉTAPE 2 : IMDb BASICS (Filtre >= 1960)
# ==============================================================================
print("\n2️⃣  IMDb Basics : Sélection (>= 1960)...")
url_basics = "https://datasets.imdbws.com/title.basics.tsv.gz"
chunks_basics = []

if 'imdb_id' in df_tmdb.columns:
    ids_tmdb = set(df_tmdb['imdb_id'].dropna())
else:
    ids_tmdb = set()

with pd.read_csv(url_basics, sep='\t', compression='gzip', 
                 usecols=['tconst', 'titleType', 'startYear', 'isAdult', 'primaryTitle'], 
                 chunksize=100000) as reader:
    for chunk in reader:
        chunk['startYear'] = pd.to_numeric(chunk['startYear'], errors='coerce')
        chunk['isAdult'] = pd.to_numeric(chunk['isAdult'], errors='coerce').fillna(0)
        
        # --- C'EST ICI QUE J'AI FAIT LE CHANGEMENT ---
        mask = (
            (chunk['titleType'] == 'movie') & 
            (chunk['isAdult'] == 0) & 
            (chunk['startYear'] >= 1960) &   # <--- Correction : Année >= 1960
            (chunk['tconst'].isin(ids_tmdb))
        )
        
        res = chunk[mask]
        if not res.empty:
            chunks_basics.append(res[['tconst', 'primaryTitle', 'startYear']])

df_basics = pd.concat(chunks_basics)
print(f"   ✅ Films retenus (1960-2025) : {len(df_basics)}")


# ==============================================================================
# ÉTAPE 3 : IMDb DIRECTORS
# ==============================================================================
print("\n3️⃣  IMDb Directors...")
url_principals = "https://datasets.imdbws.com/title.principals.tsv.gz"
chunks_directors = []
ids_films_finaux = set(df_basics['tconst'])

with pd.read_csv(url_principals, sep='\t', compression='gzip', 
                 usecols=['tconst', 'nconst', 'category'], chunksize=500000) as reader:
    for chunk in reader:
        mask = (chunk['category'] == 'director') & (chunk['tconst'].isin(ids_films_finaux))
        if not chunk[mask].empty:
            chunks_directors.append(chunk[mask][['tconst', 'nconst']])

if chunks_directors:
    df_directors = pd.concat(chunks_directors).drop_duplicates(subset='tconst')
    df_basics = pd.merge(df_basics, df_directors, on='tconst', how='left')


# ==============================================================================
# ÉTAPE 4 : IMDb AKAS (Régions)
# ==============================================================================
print("\n4️⃣  IMDb Akas (Régions)...")
url_akas = "https://datasets.imdbws.com/title.akas.tsv.gz"
chunks_akas = []

with pd.read_csv(url_akas, sep='\t', compression='gzip', 
                 usecols=['titleId', 'region', 'language'], chunksize=100000) as reader:
    for chunk in reader:
        mask = chunk['titleId'].isin(ids_films_finaux)
        if not chunk[mask].empty:
            chunks_akas.append(chunk[mask].dropna(subset=['region']))

if chunks_akas:
    df_akas = pd.concat(chunks_akas).drop_duplicates(subset='titleId')
    df_basics = pd.merge(df_basics, df_akas, left_on='tconst', right_on='titleId', how='left')


# ÉTAPE 5 : FUSION FINALE
print("FUSION FINALE...")
df_final = pd.merge(df_basics, df_tmdb, left_on='tconst', right_on='imdb_id', how='inner')
df_final = df_final.drop(columns=['titleId', 'imdb_id'])

print(f"🎉 RÉSULTAT FINAL : {len(df_final)} films (1960+).")
display(df_final.head())

# Export
df_final.to_csv("Dataset_1960_Plus.csv", index=False)

🚀 DÉMARRAGE : Mode Strict (Films >= 1960)...

1️⃣  TMDB : Chargement et Nettoyage...
   -> Fichier déjà là.


C:\Users\kinga\AppData\Local\Temp\ipykernel_23016\3572125061.py:23: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmdb = pd.read_csv(output_tmdb)



2️⃣  IMDb Basics : Sélection (>= 1960)...


KeyboardInterrupt: 